In [ ]:
import pyrallis
from dataclasses import dataclass, field
import torch
import torch.nn as nn
from scipy.linalg import sqrtm
from tqdm import trange

In [ ]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
@dataclass
class TrainConfig:
    # wandb params
    project: str = "mocha"
    name: str = "test_run"
    # training
    dataset_path: str = "data"
    folder_path: str = "fmtl_small_data"
    ntrials: int = 1
    training_percent: float = 0.75
    # mocha
    w_update: bool = False
    inner_iters: int = 100
    outer_iters: int = 100
    type: str = "C"
    avg: bool = True
    lambd: float = 10
    mocha_sdca_frac: float = 0.001

In [ ]:
def load_data(path):
    raise NotImplementedError


def compute_primal(X, Y, W, Omega, lambd):
    total_loss = 0
    m = len(X)
    for t in range(m):
        preds = Y[t] * (W[:, t] @ X[t])
        total_loss = total_loss + torch.mean(torch.maximum(torch.tensor(0), 1.0 - preds))
    primal_obj = total_loss + lambd / 2 * torch.trace(W @ Omega @ W.T)
    return primal_obj


def compute_dual(alpha, Y, W, Omega, lambd):
    total_alpha = 0
    m = len(Y)
    for tt in range(m):
        total_alpha = total_alpha + torch.mean(-1.0 * alpha[tt] * Y[tt])
    dual_obj = -lambd / 2 * torch.trace(W @ Omega @ W.T)
    return dual_obj


# TODO: break down into predict and compute
def compute_rmse(X, Y, W, type="C", avg=True):
    m = len(X)
    Y_hat = []
    for t in range(m):
        # regression
        if type == "R":
            Y_hat.append(W[:, t] @ X[t])
        # classification
        else:
            Y_hat.append(torch.sign(W[:, t] @ X[t]))

    if avg:
        all_errs = torch.zeros(m, device=DEVICE)
        for t in range(m):
            if type == "R":
                all_errs[t] = torch.sqrt(torch.mean((Y[t] - Y_hat[t]) ** 2))
            else:
                all_errs[t] = torch.mean(torch.abs(Y[t] - Y_hat[t]) / 2)
        err = torch.mean(all_errs)
    else:
        Y = torch.vstack(Y)
        Y_hat = torch.vastack(Y_hat)
        if type == "R":
            err = torch.sqrt(torch.mean((Y - Y_hat) ** 2))
        else:
            err = torch.mean(torch.abs(Y - Y_hat))
    return err

In [ ]:
(torch.zeros(5) @ torch.zeros(5, 10)).shape

# data loading section

In [ ]:
import numpy as np
import re

In [ ]:
def parce_har(path):
    data = []
    with open(path) as f:
        for line in f:
            features = line.strip().split()
            data.append(features)
    return np.asarray(data, dtype=np.float32)

In [ ]:
train_folder_path = "/Users/agarkov/Downloads/human+activity+recognition+using+smartphones/UCI HAR Dataset/train/"
X_train_path = train_folder_path + "X_train.txt"
y_train_path = train_folder_path + "y_train.txt"

test_folder_path = "/Users/agarkov/Downloads/human+activity+recognition+using+smartphones/UCI HAR Dataset/test/"
X_test_path = test_folder_path + "X_test.txt"
y_test_path = test_folder_path + "y_test.txt"

In [ ]:
X_train = parce_har(X_train_path)
Y_train = parce_har(y_train_path)

X_test = parce_har(X_test_path)
Y_test = parce_har(y_test_path)

In [ ]:
# dummy func to get arrays
def dummy_convert(X, y, min_size, max_size, num_datasets):
    Xs, ys = [], []
    size = y.shape[0]
    for t in range(num_datasets):
        curr_size = np.random.randint(low=min_size, high=max_size+1)
        idxs = np.random.choice(np.arange(size), curr_size, replace=False)
        # [num_features, batch_size]
        Xs.append(torch.tensor(X[idxs].T, device=DEVICE))
        ys.append(torch.tensor(y[idxs], device=DEVICE))
        
    return Xs, ys

In [ ]:
X_train.shape

# in case we have mulitiple files representing nodes

In [ ]:
import os

In [ ]:
def comma_parce_har(path):
    data = []
    with open(path) as f:
        for line in f:
            features = line.strip().split(',')
            data.append(features)
    return np.asarray(data, dtype=np.float32)

In [ ]:
def files_parser(path: str):
    X_files = []
    Y_files = []
    for subdir, _, files in os.walk(path):
        for file in files:
            if file.startswith("X"):
                X_files.append(file)
            if file.startswith("Y"):
                Y_files.append(file)
    
    X = []
    Y = []
    X_files.sort()
    Y_files.sort()
        
    for curr_X, curr_Y in zip(X_files, Y_files):
        # in case X_i.txt and Y_i.txt namings
        assert curr_X[1:] == curr_Y[1:]
        X.append(torch.tensor(comma_parce_har(os.path.join(path, curr_X)).T))
        Y.append(torch.tensor(comma_parce_har(os.path.join(path, curr_Y))))
        assert X[-1].shape[-1] == Y[-1].shape[0]
        
    return X, Y

In [ ]:
X, Y = files_parser("fmtl_small_data/")

# running train section

In [ ]:
# @pyrallis.wrap()
def train(cfg: TrainConfig):
    # Xtrain = [Xtrain_1, Xtrain_2, ..., Xtrain_m] data from m nodes
    # Xtrain_i = [num_features, batch_size_i]
    Xtrain, Ytrain, Xtest, Ytest = files_parser(cfg.folder_path)
    
    m = len(Xtrain)
    d = Xtrain[0].shape[0]
    W = torch.zeros(d, m, device=DEVICE)
    alpha = []
    Sigma = torch.eye(m, device=DEVICE) / m
    Omega = torch.inverse(Sigma)
    totaln = 0
    n = np.zeros(m, dtype=np.int32)
    loss = []
    for t in range(m):
        n[t] = Ytrain[t].shape[0]
        totaln += n[t]
        alpha.append(torch.zeros(n[t], device=DEVICE))

    rho = 1
    if cfg.w_update:
        rmse = torch.zeros(cfg.inner_iters)
        dual_objs = torch.zeros(cfg.inner_iters)
        primal_objs = torch.zeros(cfg.inner_iters)
    else:
        rmse = torch.zeros(cfg.outer_iters)
        dual_objs = torch.zeros(cfg.outer_iters)
        primal_objs = torch.zeros(cfg.outer_iters)

    for h in trange(cfg.outer_iters):
        if not cfg.w_update:
            curr_err = compute_rmse(Xtest, Ytest, W, type="C", avg=True)
            rmse[h] = curr_err
            loss.append(curr_err)
            print(curr_err)
            primal_objs[h] = compute_primal(Xtrain, Ytrain, W, Omega, cfg.lambd)
            dual_objs[h] = compute_dual(alpha, Ytrain, W, Omega, cfg.lambd)

        for hh in range(cfg.inner_iters):
            # TODO: set rng here
            if cfg.w_update:
                rmse[hh] = compute_rmse(Xtest, Ytest, W, type="C", avg=True)
                primal_objs[hh] = compute_primal(Xtrain, Ytrain, W, Omega, cfg.lambd)
                dual_objs[hh] = compute_dual(alpha, Ytrain, W, Omega, cfg.lambd)
                
            deltaW = torch.zeros((d, m), device=DEVICE)
            deltaB = torch.zeros((d, m), device=DEVICE)
            for t in range(m):
                tperm = torch.randperm(n[t])
                alpha_t = alpha[t]
                curr_sig = Sigma[t, t]
                local_iters = int(n[t] * cfg.mocha_sdca_frac)
                
                curr_err = compute_rmse(Xtest, Ytest, W, type="C", avg=True)
                loss.append(curr_err)
                print(curr_err)

                for s in range(local_iters):
                    # select random coordinate
                    idx = tperm[s % n[t]]
                    alpha_old = alpha_t[idx].clone()
                    curr_y = Ytrain[t][idx]
                    curr_x = Xtrain[t][:, idx]

                    # compute update
                    update = curr_y * curr_x @ (W[:, t] + rho * deltaW[:, t])
                    grad = cfg.lambd * n[t] * (1.0 - update) / (
                        curr_sig * rho * (curr_x.T @ curr_x)
                    ) + (alpha_old * curr_y)
                    alpha_t[idx] = curr_y * torch.maximum(torch.tensor(0.0), torch.minimum(torch.tensor(1.0), grad))
                    deltaW[:, t] += Sigma[t, t] * (alpha_t[idx] - alpha_old) * curr_x / (cfg.lambd * n[t])
                    deltaB[:, t] += (alpha_t[idx] - alpha_old) * curr_x / n[t]
                    alpha[t] = alpha_t
            for t in range(m):
                for tt in range(m):
                    W[:, t] += deltaB[:, tt] * Sigma[t, tt] / cfg.lambd

        
        
        # make sure eigenvaluers are positive
        A = W.T @ W
        if torch.any(torch.linalg.eigvals(A).real < 0):
            Dmat, V = torch.linalg.eig(A)
            Dmat, V = Dmat.real, V.real
            Dmat[Dmat <= 1e-7] = 1e-7
            D_c = torch.diag(Dmat)
            A = (V @ D_c @ V.T)

        sqm = torch.tensor(sqrtm(A).real, device=DEVICE)
        Sigma = sqm / torch.trace(sqm)
        Omega = torch.linalg.inv(Sigma)
        rho = torch.max(torch.sum(torch.abs(Sigma), dim=1) / torch.diag(Sigma))
        
    return W, loss

Lib SVM + гетерогенность данных

Разбить по PCA датасет (с одним распределением) и замешать по преимущественным распределением

Можно на сгенерированных датасетах.


# mushrooms dataset

In [ ]:
dataset = "mushrooms.txt" 
#файл должен лежать в той же деректории, что и notebook

In [ ]:
from sklearn.datasets import load_svmlight_file
data = load_svmlight_file(dataset)
X, y = data[0].toarray(), data[1]

In [ ]:
y = 2 * y - 3

In [ ]:
mush_nodes = []
mush_nodes_y = []
num_nodes = 10
size = X.shape[0] // num_nodes
for i in range(num_nodes):
    mush_nodes.append(torch.tensor(X[i*size:(i+1)*size].T, dtype=torch.float32))
    mush_nodes_y.append(torch.tensor(y[i*size:(i+1)*size], dtype=torch.float32))

In [ ]:
def files_parser(folder_path):
    return train_x, train_y, test_x, test_y

# LibSVM a9a file

In [ ]:
# download train part
# !curl -o a9a.txt https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/binary/a9a

In [ ]:
# download test part
# !curl -o a9a_test.txt https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/binary/a9a.t

In [ ]:
dataset = "a9a.txt" 
#файл должен лежать в той же деректории, что и notebook

In [ ]:
data = load_svmlight_file(dataset)
X, y = data[0].toarray(), data[1]

In [ ]:
mush_nodes = []
mush_nodes_y = []
num_nodes = 10
size = X.shape[0] // num_nodes
for i in range(num_nodes):
    mush_nodes.append(torch.tensor(X[i*size:(i+1)*size].T, dtype=torch.float32))
    mush_nodes_y.append(torch.tensor(y[i*size:(i+1)*size], dtype=torch.float32))

# linreg baseline

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score as acc

In [ ]:
model = LinearRegression()
model.fit(X, y)
preds = model.predict(X) > 0
acc(preds, y)

# LibSVM w8 file

In [ ]:
# download train part
!curl -o w8a.txt https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/binary/w8a

In [ ]:
# download test part
!curl -o w8a_test.txt https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/binary/w8a.t

In [ ]:
train_file = "w8a.txt" 
test_file = "w8a_test.txt" 
#файл должен лежать в той же деректории, что и notebook

In [ ]:
train = load_svmlight_file(train_file)
train_x, train_y = train[0].toarray(), train[1]

test = load_svmlight_file(test_file)
test_x, test_y = test[0].toarray(), test[1]

train_test_x = np.vstack((train_x, test_x))
train_test_y = np.concatenate((train_y, test_y))

# LibSVM phishing dataset

In [ ]:
!curl -o phishing.txt https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/binary/phishing

In [ ]:
train_file = "phishing.txt" 
#файл должен лежать в той же деректории, что и notebook

In [ ]:
train = load_svmlight_file(train_file)
train_x, train_y = train[0].toarray(), train[1]

train_test_x = train_x
train_test_y = train_y

# K-means for data heterogenity

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
num_clusters = 4
kmeans = KMeans(n_clusters=num_clusters)
kmeans.fit(train_test_x)

In [ ]:
train_part = 0.8

clust_train_x = []
clust_train_y = []

clust_test_x = []
clust_test_y = []

for i in range(num_clusters):
    idxs = kmeans.labels_ == i
    
    curr_x = train_test_x[idxs]
    curr_y = train_test_y[idxs]
    
    size = curr_y.size
    
    curr_train_x = curr_x[:int(train_part*size)]
    curr_train_y = curr_y[:int(train_part*size)]
    
    curr_test_x = curr_x[int(train_part*size):]
    curr_test_y = curr_y[int(train_part*size):]
    
    clust_train_x.append(torch.tensor(curr_train_x.T, dtype=torch.float32))
    clust_train_y.append(torch.tensor(curr_train_y, dtype=torch.float32))
    
    clust_test_x.append(torch.tensor(curr_test_x.T, dtype=torch.float32))
    clust_test_y.append(torch.tensor(curr_test_y, dtype=torch.float32))

In [ ]:
def files_parser(folder_path):
    return clust_train_x, clust_train_y, clust_test_x, clust_test_y

In [ ]:
# train
config = TrainConfig()
for lambd in [0.00001]:
    config.lambd = lambd
    W, rmse = train(config)